In [ ]:
import pandas as pd
import re, ast
import psycopg2


def get_con_data():  # Подключение к бд bi
    print('Подключение к БД...')
    doc = open(r'connection str BI.txt')
    connect = doc.read()
    connect = ast.literal_eval(re.sub(r'\n','', connect))
    return connect


def get_table_num(con_data, table_name):    # Получение названия таблиц из бд bi
    con = psycopg2.connect(
        database = con_data['database'], 
        user = con_data['user'], 
        password = con_data['password'], 
        host = con_data['host'], 
        port = con_data['port'],
        options = con_data['options'])

    with con.cursor() as cur:
        cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';") 
        tables = cur.fetchall()
        cur.close()

    df0 = pd.DataFrame(tables)
    df01 = df0[df0[0].str.contains(table_name)].sort_values(by = [0], ascending=False)[:1]
    table_num = df01.to_string(index = False).split(table_name, 1)[1]  # автоматическое получение номера таблицы, так как они меняются с определенной периодичностью

    return table_num



if __name__=="__main__":
    CONNECT = get_con_data()

    con = psycopg2.connect(
            database = CONNECT['database'], 
            user = CONNECT['user'], 
            password = CONNECT['password'], 
            host = CONNECT['host'], 
            port = CONNECT['port'],
            options = CONNECT['options'])

    df = pd.read_sql(
        f"""
            SELECT t.rieltor_city, 
                t.ticket_id, 
                (case 
                    when LENGTH(cl.phone) = 10 and left(cl.phone, 1) = '9' 
                    then concat(8, cl.phone) else cl.phone end
                ) "phone",
                t.created_ticket, t.closed_ticket, t.close_status

            from source_voronka_ticket_buyer_test_v{get_table_num(CONNECT, 'source_voronka_ticket_buyer_test_v')} t
                left join source_client_ticket_v{get_table_num(CONNECT, 'source_client_ticket_v')} cl on cl.client_id = t.client_id
                left join source_cities_v{get_table_num(CONNECT, 'source_cities_v')} c on c.city_id = t.city_rielt_id
                
            where t.department = 'Отдел продаж'
                and t.type in ('Покупка вторичной', 'Покупка загородной')
                and t.u_fio is not null and t.u_fio != 'null' and lower(t.u_fio) not like ALL(ARRAY['test', 'тест', 'риэс', 'ries'])
                and t.created_ticket >= '2023-12-01'  /* период для теста, потом убрать*/
                and cl.phone is not null and left(cl.phone, 1) != '0'
                and cl.fio_client not like ALL(ARRAY['test', 'тест', 'риэс', 'ries'])
                and cl.phone ~ '^[0-9]+$'

                /* выбрана только Россия, добавить другие страны и прописать в select правила обработки и унификации тел номеров, учесть сотовые и стационарные номера */
                and (c.country_id = 1     
                    and length(cl.phone) >= 10)
        """,
        con = con
    )
